In [10]:
import numpy as np
import pandas as pd
import itertools
import csv
import nltk

In [11]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [12]:
###Extracting sentences from csv file
with open("reddit.csv", "r", encoding="utf-8") as file:
    read = csv.reader(file, skipinitialspace = True)
    next(read)
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in read])
    sentences = ["%s %s %s"%(sentence_start_token,x,sentence_end_token) for x in sentences]
    print("number of parsed sentences {}".format(len(sentences)))
        

number of parsed sentences 79170


In [13]:
###Vocabulary info
wordtokens = nltk.FreqDist(itertools.chain(*[nltk.word_tokenize(x) for x in sentences]))

In [14]:
###most common words
vocab = wordtokens.most_common(vocabulary_size-1)

In [15]:
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [16]:
print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

Using vocabulary size 8000.
The least frequent word in our vocabulary is 'documentary' and appeared 10 times.


In [17]:
###replacing words not in vocab as unknown_token
tokenised_sent = [nltk.word_tokenize(x) for x in sentences]

In [18]:
for i,sent in enumerate(tokenised_sent):
    tokenised_sent[i] = [w if w in index_to_word else unknown_token for w in sent]

In [19]:
print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenised_sent[0])


Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'


In [20]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenised_sent])
Y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenised_sent])

Equations:
         $s_t = \tanh(Ux_t + Ws_{t-1})$  
         $o_t = softmax(Vs_t)$
dimensions:
        - x_t: 8000 x 1
        - U: 100 x 8000
        - W: 100 x 100
        - V: 8000 x 100
        - o_t: 8000 x 1
        - s_t: 100 x 1

In [2]:
class RNN_numpy:
    def __init__(self, word_dim, hidden_dim):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        ##initialising parameters
        self.U = np.random.normal(size = (hidden_dim, word_dim))
        self.W = np.random.normal(size = (hidden_dim, hidden_dim))
        self.V = np.random.normal(size = (word_dim, hidden_dim))
        
        def softmax(self,array):
            return np.exp(array) / np.sum(np.exp(array), axis=0)
        
        ###method for forward propagation
        def forward_propagation(self, x):
            ##number of time steps
            T = len(x)
            s = np.zeros((T+1, self.hidden_dim))
            o = np.zeros(T, self.word_dim)
            
            for t in range(T):
                s[t] = np.tanh(self.U[:,x[t]] + np.dot(self.W, s[t-1])) ##(100 x 8000) . (8000 x 1) + (100 x 100) . (100 x 1) = (100 x 1)
                o[t] = softmax(np.dot(self.V, s[t]))                    ##(8000 x 100) + (100 x 1) =(8000 x 1)
            
            return [o,s]
            

(20,)